# Tugas 5

## Scrapping Data Comment Youtube

In [ ]:
!pip install selenium --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Scrap

In [ ]:

data=[]

with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
    wait = WebDriverWait(driver,10)
    driver.get("https://www.youtube.com/watch?v=gKZg9E8_kCY")

    for item in range(10): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7220\2069022387.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:


In [ ]:
import pandas as pd   
df = pd.DataFrame(data, columns=['comment'])
df

,comment
0,"ID\nAbaikan navigasi\nLogin\n0:00 / 16:11\n""Ko..."
1,
2,Semoga masyarakat Indonesia bisa cerdas dalam ...
3,"Siapapun presiden yg terpilih ga jadi masalah,..."
4,"Ya Allah, mudah-mudahan Indonesia punya Presid..."
...,...
204,
205,
206,
207,


### Prepocessing and cleaning

In [ ]:
import re, string
import numpy as np

# Text Cleaning
def cleaning(text):
    # HTML Tag Removal

    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [ ]:
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang, emoji_to_words, remove_html])
# steamming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
df['comment (clean)'] = df['comment'].apply(lambda x: pipe(x))
df['comment (clean)'] = df['comment (clean)'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['comment (clean)'] = df['comment (clean)'].replace('', np.nan)


In [ ]:
print(df.isna().sum())

comment            0
comment (clean)    8
dtype: int64


In [ ]:
df.dropna(inplace=True)

### Data clean 

In [ ]:
df

,comment,comment (clean)
0,"ID\nAbaikan navigasi\nLogin\n0:00 / 16:11\n""Ko...",id abaikan navigasi login koalisinya tapi capr...
2,Semoga masyarakat Indonesia bisa cerdas dalam ...,semoga masyarakat indonesia bisa cerdas dalam ...
3,"Siapapun presiden yg terpilih ga jadi masalah,...",siapapun presiden yang terpilih enggak jadi ma...
4,"Ya Allah, mudah-mudahan Indonesia punya Presid...",ya allah mudah mudahan indonesia punya preside...
5,"ya Allah lindungilah anak cucu kami, NKRI dari...",ya allah lindungilah anak cucu kami nkri dari ...
...,...,...
197,Koruptor satu ini kok gak malu nongol di tv te...,koruptor satu ini kok enggak malu nongol di tv...
198,Ganjar adalah pilihan🇮🇩🇮🇩🇮🇩🇮🇩,ganjar adalah pilihan bendera indonesia bender...
199,Gaspoll GANJAR,gaspol ganjar
200,Ganjar bersih transparan akuntabel dan profesi...,ganjar bersih transparan akuntabel dan profesi...


### Export

In [ ]:
# to csv
df.to_csv('datacomment_cnn_fix.csv', index=False)

## LSA Topic Modeling

### Read Data

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/Much-rafi/data/main/datacomment.csv')
df

,comment,comment (clean)
0,"ID\nAbaikan navigasi\nLogin\n0:00 / 16:11\n""Ko...",id abaikan navigasi login koalisinya tapi capr...
1,Semoga masyarakat Indonesia bisa cerdas dalam ...,semoga masyarakat indonesia bisa cerdas dalam ...
2,"Siapapun presiden yg terpilih ga jadi masalah,...",siapapun presiden yang terpilih enggak jadi ma...
3,"Ya Allah, mudah-mudahan Indonesia punya Presid...",ya allah mudah mudahan indonesia punya preside...
4,"ya Allah lindungilah anak cucu kami, NKRI dari...",ya allah lindungilah anak cucu kami nkri dari ...
...,...,...
196,Koruptor satu ini kok gak malu nongol di tv te...,koruptor satu ini kok enggak malu nongol di tv...
197,Ganjar adalah pilihan🇮🇩🇮🇩🇮🇩🇮🇩,ganjar adalah pilihan bendera indonesia bender...
198,Gaspoll GANJAR,gaspol ganjar
199,Ganjar bersih transparan akuntabel dan profesi...,ganjar bersih transparan akuntabel dan profesi...


### Modeling

In [ ]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

### Bobot Kata

In [ ]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('ganjar', 0.5307488621004811), ('rakyat', 0.40372223178772954)]
Topic 1:  [('ganjar', 0.6266324857105154), ('dukung', 0.18023780802075773)]
Topic 2:  [('adian', 0.7413117063512099), ('bang', 0.4650312960377806)]
Topic 3:  [('indonesia', 0.48864672573628254), ('bendera', 0.36976490165668396)]


### Bobot setiap topik terhadap dokumen

In [ ]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id abaikan navigasi login koalisinya tapi capr...,0.397869,0.001811,0.076265,0.013301,0
1,semoga masyarakat indonesia bisa cerdas dalam ...,0.410896,-0.372691,-0.014145,0.168993,0
2,siapapun presiden yang terpilih enggak jadi ma...,0.296135,-0.274659,-0.069541,0.484561,3
3,ya allah mudah mudahan indonesia punya preside...,0.246081,-0.202269,-0.069339,0.273631,3
4,ya allah lindungilah anak cucu kami nkri dari ...,0.063108,-0.036666,-0.014331,0.051846,0
...,...,...,...,...,...,...
196,koruptor satu ini kok enggak malu nongol di tv...,0.022947,0.008059,-0.001712,-0.009137,0
197,ganjar adalah pilihan bendera indonesia bender...,0.283062,-0.157622,-0.059323,0.565706,3
198,gaspol ganjar,0.256992,0.338052,-0.041040,0.009679,1
199,ganjar bersih transparan akuntabel dan profesi...,0.193686,0.223441,-0.011776,-0.008784,1
